In [ ]:
include("../../QR/housQR.jl")
include("../../utilities/genFunc.jl")
using .housQR
using LinearAlgebra, BenchmarkTools, CSV, DataFrames

baseDir = joinpath("../", "results/QR")
mkpath(baseDir);

In [ ]:
### time with respect to lambda

# parameters
lambdas = 10. .^(-15:6)
epss    = [nothing]
m       = 300
n       = 20


# output csv
outputvsc = joinpath(baseDir, "statisticsQR-lambda-m" * string(m) * "n" * string(n) * "--time.csv");
accData = Dict(
    :lambda   => Array{Float64}(undef, 0),
    :meantime => Array{Float64}(undef, 0),
    :stdtime  => Array{Float64}(undef, 0)
    )

# computation
for (λ, ϵ) ∈ Iterators.product(lambdas, epss) |> collect
    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)

    t = @benchmark begin
        QR = qrfact($gf[:X_hat])
        w = QR \ $gf[:y_hat]
    end

    push!(accData[:lambda], λ)
    push!(accData[:meantime], mean(t.times))
    push!(accData[:stdtime], std(t.times))
    println("Done: λ " * string(λ))
    flush(stdout)
end

CSV.write(outputvsc, DataFrame(accData));

In [ ]:
### error with respect to lambda

# parameters
lambdas = 10. .^(-15:6)
epss    = [nothing]
m       = 300
n       = 20


# output csv
outputvsc = joinpath(baseDir, "statisticsQR-lambda-m" * string(m) * "n" * string(n) * "--error.csv");
accData = Dict(
    :lambda   => Array{Float64}(undef, 0),
    :relative => Array{Float64}(undef, 0),
    :residual => Array{Float64}(undef, 0),
    :stability => Array{Float64}(undef, 0)
    )

# computation
for (λ, ϵ) ∈ Iterators.product(lambdas, epss) |> collect
    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)

    QR = qrfact(gf[:X_hat])
    w = QR \ gf[:y_hat]
    
    relative_error = norm(w - gf[:w_star]) / norm(gf[:w_star])
    residual = norm(gf[:X_hat] * w - gf[:y_hat]) / norm(gf[:y_hat])
    stability = norm(Q_times_R - gf[:X_hat]) / norm(gf[:X_hat])

    push!(accData[:lambda], λ)
    push!(accData[:relative], relative_error)
    push!(accData[:residual], residual)
    push!(accData[:stability], stability)
    println("Done: λ " * string(λ))
    flush(stdout)
end

CSV.write(outputvsc, DataFrame(accData));

In [ ]:
### error with respect to lambda

# parameters
lambdas = 10. .^(-15:6)
epss    = [nothing]
m       = 300
n       = 20
perturbation = 1e-10


# output csv
outputvsc = joinpath(baseDir, "statisticsQR-forward-m" * string(m) * "n" * string(n) * "--error.csv");
accData = Dict(
    :lambda   => Array{Float64}(undef, 0),
    :forwardQ => Array{Float64}(undef, 0),
    :forwardR => Array{Float64}(undef, 0),
    )

# computation
for (λ, ϵ) ∈ Iterators.product(lambdas, epss) |> collect
    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)

    QR = qrfact(gf[:X_hat])
    w = QR \ gf[:y_hat]

    X_hat_p = gf[:X_hat] + perturbation * randn(size(gf[:X_hat]))
    cond(gf[:X_hat]) |> display
    # cond(X_hat_p) |> display
    QR_p = qrfact(X_hat_p)

    forwardQ = norm(QR.Q - QR_p.Q)
    forwardR = norm(QR.R - QR_p.R) / norm(QR.R)

    push!(accData[:lambda], λ)
    push!(accData[:forwardQ], forwardQ)
    push!(accData[:forwardR], forwardR)

    println("Done: λ " * string(λ))
    flush(stdout)
end

CSV.write(outputvsc, DataFrame(accData));